### Imports

In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support import expected_conditions as EC

### Automation with Selenium

In [ ]:
driver = webdriver.Chrome()
url = "https://www.imdb.com/"
delay = 3
#open IMDB
driver.get(url)

#maximize windows
driver.maximize_window()

# select the dropdown list
dropdown = driver.find_element(By.CLASS_NAME, "ipc-icon--arrow-drop-down")
dropdown.click()

# click on advanced search
WebDriverWait(driver, delay).until(EC.presence_of_element_located((By.LINK_TEXT, "Advanced Search")))
advancedSearch = driver.find_element(By.LINK_TEXT, "Advanced Search")
advancedSearch.click()

# Click on advanced title search
WebDriverWait(driver, delay).until(EC.presence_of_element_located((By.LINK_TEXT, "Advanced Title Search")))
adv_title = driver.find_element(By.LINK_TEXT, "Advanced Title Search")
adv_title.click()

# Click on Feature Film, TV movie
WebDriverWait(driver, delay).until(EC.presence_of_element_located((By.ID, "header")))
featureFilm = driver.find_element(By.ID, "title_type-1")
featureFilm.click()
tvMovie = driver.find_element(By.ID, "title_type-2")
tvMovie.click()

# Release Dates min and max
min_date = driver.find_element(By.NAME, "release_date-min")
min_date.click()
min_date.send_keys("1950")
max_date = driver.find_element(By.NAME, "release_date-max")
max_date.click()
max_date.send_keys("2020")

#user rating
minRating = driver.find_element(By.NAME, "user_rating-min")
minRating.click()
min_dropdown = Select(minRating)
min_dropdown.select_by_visible_text("7.0")

maxRating = driver.find_element(By.NAME, "user_rating-max")
maxRating.click()
max_dropdown = Select(maxRating)
max_dropdown.select_by_visible_text("10")

#oscar nominated
driver.find_element(By.ID, "groups-7").click()

#saerch count
searchCount = driver.find_element(By.ID, "search-count")
searchCount.click()
searchCount = Select(searchCount)
searchCount.select_by_index(2)

#search start
submit = driver.find_element(By.XPATH, "(//button[@type='submit'])[2]")
submit.click()

#save url
searchedURL = []

# func. to find the next bottom
def check(classname):
    try:
        driver.find_element(By.CLASS_NAME, classname)
    except NoSuchElementException:
        return False
    return True

# check point for next point
while check('lister-page-next') is True:
    searchedURL.append(driver.current_url)
    driver.find_element(By.CLASS_NAME, 'lister-page-next').click()
else:
    searchedURL.append(driver.current_url)
    pass

# create empty list to append later
movie_title = []
released_year = []
run_time = []
genre = []
rating = []
vote_counts = []
director = []

#for loop to save the different urls
for url in searchedURL:
    # get request url changed automatically
    response = requests.get(url)
    # soup object
    soup = BeautifulSoup(response.content, 'html.parser')
    #result items
    list_items = soup.findAll('div', {'class':'lister-item'})
    for lis in list_items:
        movie_title.append(lis.find('h3').find('a').get_text())
        released_year.append(lis.find('h3').find('span', {'class':'lister-item-year'}).get_text().replace(' TV Movie', '').split(' ')[-1].replace('(','').replace(')',''))
        run_time.append(lis.find('p').find('span', {'class':'runtime'}).get_text().replace(' min',''))
        genre.append(lis.find('p').find('span', {'class':'genre'}).get_text().strip())
        rating.append(lis.find('div', {'class':'ratings-bar'}).find('strong').get_text().strip())
        vote_counts.append(lis.find('p', {'class':'sort-num_votes-visible'}).find('span', {'name':'nv'}).get_text().replace(',',''))
        director.append(lis.find('p', {'class':''}).find('a').get_text())
        imdb_df = pd.DataFrame({'Movie Title':movie_title, "Released Year":released_year, "Movie Duration":run_time, "Genre":genre, "Rating":rating, "Vote Counts":vote_counts, "Director":director})

#export to csv file
imdb_df.to_csv("IMDB.csv", index=False)